In [4]:
%%capture --no-stderr
# !pip install sentence-transformers
# !pip install faiss-cpu

UsageError: unrecognized arguments: --sterror


In [5]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OllamaEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
import bs4

# Step 1: Load the document from a web url
loader = WebBaseLoader(["https://huggingface.co/blog/llama3"])
documents = loader.load()

# Step 2: Split the document into chunks with a specified chunk size
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
all_splits = text_splitter.split_documents(documents)

# Step 3: Store the document into a vector store with a specific embedding model

embedding = OllamaEmbeddings(model="llama3")
vector_db = Chroma.from_documents(all_splits, embedding=embedding, persist_directory=".")

In [6]:
from langchain_community.chat_models import ChatOllama

llm = ChatOllama(model="llama3")
llm.invoke("Hi")

AIMessage(content="Hi! It's nice to meet you. Is there something I can help you with, or would you like to chat?", response_metadata={'model': 'llama3', 'created_at': '2024-06-11T14:49:20.664012Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 684502666, 'load_duration': 2863708, 'prompt_eval_count': 10, 'prompt_eval_duration': 253829000, 'eval_count': 26, 'eval_duration': 425552000}, id='run-4ab271fe-b11f-482f-8c16-fcb5916a4a73-0')

In [8]:
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(llm=llm, retriever=vector_db.as_retriever(), return_source_documents=True)

In [9]:
output = qa.invoke("What is llama3?", return_sources=True)

output

{'query': 'What is llama3?',
 'result': 'Llama 3 is a language model developed by Meta AI. It comes in two sizes: 8B for efficient deployment and development on consumer-size GPU, and 70B for large-scale AI native applications. Both sizes come in base and instruction-tuned variants.',
 'source_documents': [Document(page_content='Llama 3 comes in two sizes: 8B for efficient deployment and development on consumer-size GPU, and 70B for large-scale AI native applications. Both come in base and instruction-tuned variants. In addition to the 4 models, a new version of Llama Guard was fine-tuned on Llama 3 8B and is released as Llama Guard 2 (safety fine-tune).', metadata={'description': 'We’re on a journey to advance and democratize artificial intelligence through open source and open science.', 'language': 'No language found.', 'source': 'https://huggingface.co/blog/llama3', 'title': "Welcome Llama 3 - Meta's new open LLM"}),
  Document(page_content='Assistant responses may end with the spe